In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
from stat_encoder import *

In [4]:
from scorer_RMSE import RMSE

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# загрузка частей train
train = pd.read_csv('train.csv',sep =',')
train2w = pd.read_csv('two_way_inter_train.csv',sep =',') 
train3w = pd.read_csv('three_way_inter_train.csv',sep =',')
train4w = pd.read_csv('four_way_inter_train.csv',sep =',')

Y = np.log( train['price'])
train['area'] = np.log( train['area']  )
train['security_deposit'] = np.log( train['security_deposit']  )
train.drop(['price','Property_ID','locality'] , axis=1 , inplace = True)

In [9]:
# загрузка частей test
test = pd.read_csv('test.csv',sep =',')
test2w = pd.read_csv('two_way_inter_test.csv',sep =',') 
test3w = pd.read_csv('three_way_inter_test.csv',sep =',')
test4w = pd.read_csv('four_way_inter_test.csv',sep =',')

test['area'] = np.log( test['area']  )
test['security_deposit'] = np.log( test['security_deposit']  )
test.drop(['price','Property_ID','locality'] , axis=1 , inplace = True)

In [10]:
train2w.columns = test2w.columns 

In [11]:
train3w.columns = test3w.columns 

In [12]:
# соединение в один датафрейм для train и test

In [13]:
full_train = pd.concat([train , train2w , train3w ,train4w] , axis=1)


In [14]:
full_test = pd.concat([test , test2w , test3w ,test4w] , axis=1)

In [15]:
#Удаление датафреймов из памяти
del test , test2w ,test3w  ,test4w 

In [16]:
del train , train2w ,train3w  ,train4w

Разделение на train , sub_validation и validation в отношении 60% , 20% , 20% соответственно

На train и sub_validation будет производиться отбор признаков и оптимизация гиперпараметров.

Затем модель обучится на (train + sub_validation ) и будет оценена на validation

Для выработки прогноза на test для посылки на сайт модель тренируется на  train + sub_validation + validation 

In [17]:
x_train , x_val , y_train , y_val = train_test_split(full_train , Y , random_state = 67 , test_size = 0.2 )

In [18]:
x_train , x_s_val , y_train , y_s_val = train_test_split(x_train , y_train , random_state = 67 , test_size = 0.25 )

In [18]:
print('Размеры выборок \n'
    ,'train ',x_train.shape , '\n' ,
      'sub_validation ',x_s_val.shape , '\n',
      'validation ',x_val.shape ,sep='')

Размеры выборок 
train (80809, 174)
sub_validation (26937, 174)
validation (26937, 174)


Отбор признаков и статистических показателей для кодирования

Нужно отобрать категориальные признаки, дающие высокий резуьтат по метрике RMSE.

К каждому категориальному признаку применяется кодирование по статистическому показателю.

Оно состоит в следующем:
1. Значения целевой переменной группируется по уровням категории (2 уровня - если признак бинарный)
2. По группам считается стат. показатель (среднее, процентили и т. п.)
3. Значения категориального признака заменяются вычисленными показателями.

Эта техника называется mean encoding.

Нужно также отобрать выгодную комбинацию стат. показателей.

In [19]:
# списов всех категориальных переменных
categorical_features =  x_train.columns[(x_train.columns !='area' ) & (x_train.columns !='security_deposit' )]  

In [20]:
len( categorical_features )

172

In [20]:
# варианты статистических показателей для кодирования.
# q - значит квантиль
stat_parameter = [
    ['mean'] ,
    ['mean', 'q50' ] ,
    ['mean'  ,'q25','q50', 'q75' , 'min','max'] ,
    ['mean'  , 'q50' , 'min','max'] ,

    ['mean' ,'q10', 'q20', 'q30', 'q40', 'q50', 'q60', 'q70', 'q80', 'q90','q25','q50', 'q75' , 'min','max'] ,
 
]

Baseline-модель - Случайный лес с парамерами по умолчанию (100 деревьев и др. параметры)

Используются все признаки и все показатели

In [23]:
# сначала производится кодирование признаков, затем обучение модели
pipe_model_baseline = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features  , stat_enc =stat_parameter[ 2],verbose = True )) ,
    ('model' , RandomForestRegressor(n_jobs =3 , verbose = 2)) ])

In [24]:
# обучение
pipe_model_baseline.fit(x_train , y_train)

Preparing feature (1/172): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/172): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/172): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/172): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/172): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/172): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/172): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/172): air_conditioner
Вычислен

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (86/172): washing_machine x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (87/172): washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (88/172): washing_machine x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (89/172): air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (90/172): air_conditioner x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (91/172): air_conditioner x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (92/172): geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (93/172): CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (94/172): room x layout_type x furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (157/172): layout_type x parking_spaces x geyser/solar x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (158/172): layout_type x power_backup x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (159/172): layout_type x washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (160/172): layout_type x air_conditioner x geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (161/172): layout_type x air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (162/172): property_type x parking_spaces x pet_friendly x washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (163/172): property_type x pet_friendly x washing_machine x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (139/172): bathroom x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (140/172): bathroom x air_conditioner x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (141/172): city x floor x air_conditioner
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (142/172): parking_spaces x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (143/172): parking_spaces x air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (144/172): parking_spaces x CCTV/security x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (145/172): washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (146/172): air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing featu

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed: 13.6min


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed: 38.5min finished


Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       ...
       'property_type x pet_friendly x washing_machine x geyser/solar',
       'furnish_type x city x parking_spaces x pet_friendly',
       'furnish_type x city x parking_spaces x lift',
       'furnish_type x c...
       'city x parking_spaces x pet_friendly x air_conditioner',
       'city x parking_spaces x washing_machine x lift',
       'city x pet_friendly x air_conditioner x lift',
       'parking_spaces x pet_friendly x power_backup x geyser/solar',
       'parking_spaces x washing_machine x CCTV/security x lift'],
      dtype='object', length=172),
                               stat_enc=['mean', 'q25', 'q50', 'q75', 'min',
                                         'max'],
                               verbose=True)),
                ('model', Rand

In [25]:
#оценка модели на наборе sub_validation
RMSE( pipe_model_baseline ,  x_s_val ,   y_s_val)

Preparing feature (1/172): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/172): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/172): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/172): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/172): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/172): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/172): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/172): air_conditioner
Вычислен

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (86/172): washing_machine x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (87/172): washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (88/172): washing_machine x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (89/172): air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (90/172): air_conditioner x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (91/172): air_conditioner x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (92/172): geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (93/172): CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (94/172): room x layout_type x furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (157/172): layout_type x parking_spaces x geyser/solar x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (158/172): layout_type x power_backup x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (159/172): layout_type x washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (160/172): layout_type x air_conditioner x geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (161/172): layout_type x air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (162/172): property_type x parking_spaces x pet_friendly x washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (163/172): property_type x pet_friendly x washing_machine x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    1.0s finished


-27602.684409930283

RMSE = 27602.68 на всех признаках с кодировкой  'mean'  ,'q25','q50', 'q75' , 'min','max' .


Оценка конвейера с признаками без взаимодействий.

In [30]:
categorical_features2 = categorical_features[:16]

In [31]:
x_train2 = x_train[categorical_features2]
x_s_val2 = x_s_val[categorical_features2]

In [32]:
pipe_model_baseline2 = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features2  , stat_enc =stat_parameter[ 2],verbose = True )) ,
    ('model' , RandomForestRegressor(n_jobs =3 , verbose = 2)) ])

In [35]:
# обучение
pipe_model_baseline2.fit(x_train2 , y_train)

Preparing feature (1/16): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/16): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/16): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/16): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/16): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/16): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/16): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/16): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/16): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/16): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/16): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/16): air_conditioner
Вычислены статистики

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   44.5s


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:  1.8min finished


Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       'washing_machine', 'air_conditioner', 'geyser/solar', 'CCTV/security',
       'lift', 'neighbourhood'],
      dtype='object'),
                               stat_enc=['mean', 'q25', 'q50', 'q75', 'min',
                                         'max'],
                               verbose=True)),
                ('model', RandomForestRegressor(n_jobs=3, verbose=2))])

In [36]:
RMSE( pipe_model_baseline2 ,  x_s_val2 ,   y_s_val)

Preparing feature (1/16): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/16): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/16): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/16): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/16): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/16): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/16): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/16): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/16): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/16): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/16): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/16): air_conditioner
Вычислены статистики

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.6s finished


-63528.28438748737

RMSE на наборе без взаимодействий равно 63528.28 - это значит, что взаимодействия признаков сильно влияют на результат обучения (разница составляет 35925.6).

Проверим, даст ли прирост в точности добавка дополнительных квантилей.

In [39]:
pipe_model_baseline3 = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features2  , stat_enc =stat_parameter[ -1],verbose = True )) ,
    ('model' , RandomForestRegressor(n_jobs =3 , verbose = 2)) ])

In [41]:
pipe_model_baseline3.fit(x_train2 , y_train)

Preparing feature (1/16): room
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (2/16): layout_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (3/16): property_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (4/16): furnish_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (5/16): bathroom
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (6/16): city
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (7/16): parking_spaces
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (8/16): floor
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (9/16): pet_friendly
Вы

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.4min


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:  4.1min finished


Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       'washing_machine', 'air_conditioner', 'geyser/solar', 'CCTV/security',
       'lift', 'neighbourhood'],
      dtype='object'),
                               stat_enc=['mean', 'q10', 'q20', 'q30', 'q40',
                                         'q50', 'q60', 'q70', 'q80', 'q90',
                                         'q25', 'q50', 'q75', 'min', 'max'],
                               verbose=True)),
                ('model', RandomForestRegressor(n_jobs=3, verbose=2))])

In [42]:
RMSE( pipe_model_baseline3 ,  x_s_val2 ,   y_s_val)

Preparing feature (1/16): room
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (2/16): layout_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (3/16): property_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (4/16): furnish_type
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (5/16): bathroom
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (6/16): city
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (7/16): parking_spaces
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (8/16): floor
Вычислены статистики: mean q10 q20 q30 q40 q50 q60 q70 q80 q90 q25 q50 q75 min max 
Preparing feature (9/16): pet_friendly
Вы

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.7s finished


-63532.04060692671

RMSE составило 63532.04 это больше чем без добавления дополнительных квантилей (на 3.76).

Выводы:
Нужно использовать взаимодействия признаков.
И набор статистических показателей:

In [22]:
# сначала производится кодирование признаков, затем обучение модели
pipe_model_baseline4 = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features  , stat_enc =stat_parameter[ 0],verbose = True )) ,
    ('model' , RandomForestRegressor(n_jobs =3 , verbose = 2)) ])

In [23]:
pipe_model_baseline4.fit(x_train , y_train)

Preparing feature (1/172): room
Вычислены статистики: mean 
Preparing feature (2/172): layout_type
Вычислены статистики: mean 
Preparing feature (3/172): property_type
Вычислены статистики: mean 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean 
Preparing feature (5/172): bathroom
Вычислены статистики: mean 
Preparing feature (6/172): city
Вычислены статистики: mean 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean 
Preparing feature (8/172): floor
Вычислены статистики: mean 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean 
Preparing feature (10/172): power_backup
Вычислены статистики: mean 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean 
Preparing feature (12/172): air_conditioner
Вычислены статистики: mean 
Preparing feature (13/172): geyser/solar
Вычислены статистики: mean 
Preparing feature (14/172): CCTV/security
Вычислены статистики: mean 
Preparing feature (15/172): lift
Вычислены статистики: mean 


Вычислены статистики: mean 
Preparing feature (105/172): layout_type x property_type x lift
Вычислены статистики: mean 
Preparing feature (106/172): layout_type x furnish_type x city
Вычислены статистики: mean 
Preparing feature (107/172): layout_type x furnish_type x power_backup
Вычислены статистики: mean 
Preparing feature (108/172): layout_type x bathroom x power_backup
Вычислены статистики: mean 
Preparing feature (109/172): layout_type x city x power_backup
Вычислены статистики: mean 
Preparing feature (110/172): layout_type x city x washing_machine
Вычислены статистики: mean 
Preparing feature (111/172): layout_type x city x air_conditioner
Вычислены статистики: mean 
Preparing feature (112/172): layout_type x parking_spaces x power_backup
Вычислены статистики: mean 
Preparing feature (113/172): layout_type x floor x pet_friendly
Вычислены статистики: mean 
Preparing feature (114/172): layout_type x floor x power_backup
Вычислены статистики: mean 
Preparing feature (115/172): la

Вычислены статистики: mean 
Preparing feature (21/172): room x city
Вычислены статистики: mean 
Preparing feature (22/172): room x parking_spaces
Вычислены статистики: mean 
Preparing feature (23/172): room x floor
Вычислены статистики: mean 
Preparing feature (24/172): room x power_backup
Вычислены статистики: mean 
Preparing feature (25/172): room x washing_machine
Вычислены статистики: mean 
Preparing feature (26/172): room x air_conditioner
Вычислены статистики: mean 
Preparing feature (27/172): room x geyser/solar
Вычислены статистики: mean 
Preparing feature (28/172): room x lift
Вычислены статистики: mean 
Preparing feature (29/172): room x neighbourhood
Вычислены статистики: mean 
Preparing feature (30/172): layout_type x property_type
Вычислены статистики: mean 
Preparing feature (31/172): layout_type x furnish_type
Вычислены статистики: mean 
Preparing feature (32/172): layout_type x bathroom
Вычислены статистики: mean 
Preparing feature (33/172): layout_type x city
Вычислены

Вычислены статистики: mean 
Preparing feature (120/172): property_type x parking_spaces x pet_friendly
Вычислены статистики: mean 
Preparing feature (121/172): property_type x parking_spaces x washing_machine
Вычислены статистики: mean 
Preparing feature (122/172): property_type x parking_spaces x air_conditioner
Вычислены статистики: mean 
Preparing feature (123/172): property_type x air_conditioner x geyser/solar
Вычислены статистики: mean 
Preparing feature (124/172): property_type x geyser/solar x CCTV/security
Вычислены статистики: mean 
Preparing feature (125/172): property_type x geyser/solar x lift
Вычислены статистики: mean 
Preparing feature (126/172): furnish_type x bathroom x geyser/solar
Вычислены статистики: mean 
Preparing feature (127/172): furnish_type x bathroom x lift
Вычислены статистики: mean 
Preparing feature (128/172): furnish_type x city x floor
Вычислены статистики: mean 
Preparing feature (129/172): furnish_type x city x power_backup
Вычислены статистики: mea

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100


building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  2.0min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:  5.9min finished


Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       ...
       'property_type x pet_friendly x washing_machine x geyser/solar',
       'furnish_type x city x parking_spaces x pet_friendly',
       'furnish_type x city x parking_spaces x lift',
       'furnish_type x c...
       'city x parking_spaces x pet_friendly x air_conditioner',
       'city x parking_spaces x washing_machine x lift',
       'city x pet_friendly x air_conditioner x lift',
       'parking_spaces x pet_friendly x power_backup x geyser/solar',
       'parking_spaces x washing_machine x CCTV/security x lift'],
      dtype='object', length=172),
                               stat_enc=['mean'], verbose=True)),
                ('model', RandomForestRegressor(n_jobs=3, verbose=2))])

In [24]:
RMSE( pipe_model_baseline4 ,  x_s_val ,   y_s_val)

Preparing feature (1/172): room
Вычислены статистики: mean 
Preparing feature (2/172): layout_type
Вычислены статистики: mean 
Preparing feature (3/172): property_type
Вычислены статистики: mean 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean 
Preparing feature (5/172): bathroom
Вычислены статистики: mean 
Preparing feature (6/172): city
Вычислены статистики: mean 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean 
Preparing feature (8/172): floor
Вычислены статистики: mean 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean 
Preparing feature (10/172): power_backup
Вычислены статистики: mean 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean 
Preparing feature (12/172): air_conditioner
Вычислены статистики: mean 
Preparing feature (13/172): geyser/solar
Вычислены статистики: mean 
Preparing feature (14/172): CCTV/security
Вычислены статистики: mean 
Preparing feature (15/172): lift
Вычислены статистики: mean 


Вычислены статистики: mean 
Preparing feature (109/172): layout_type x city x power_backup
Вычислены статистики: mean 
Preparing feature (110/172): layout_type x city x washing_machine
Вычислены статистики: mean 
Preparing feature (111/172): layout_type x city x air_conditioner
Вычислены статистики: mean 
Preparing feature (112/172): layout_type x parking_spaces x power_backup
Вычислены статистики: mean 
Preparing feature (113/172): layout_type x floor x pet_friendly
Вычислены статистики: mean 
Preparing feature (114/172): layout_type x floor x power_backup
Вычислены статистики: mean 
Preparing feature (115/172): layout_type x floor x air_conditioner
Вычислены статистики: mean 
Preparing feature (116/172): layout_type x pet_friendly x lift
Вычислены статистики: mean 
Preparing feature (117/172): layout_type x power_backup x washing_machine
Вычислены статистики: mean 
Preparing feature (118/172): layout_type x power_backup x air_conditioner
Вычислены статистики: mean 
Preparing feature 

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.7s finished


-27264.6612899378

RMSE составило 27264.66. Это на 338.02 меньше, чем при ипользовании статистик квантилей и min и max.